In [57]:
correct_dict = {
    'bank':[
        'BBVA BANCOMER',
        'BANORTE',
        'SANTANDER',
        'BANCO NACIONAL DE MEXICO',
        'BANREGIO',
        'BANAMEX',
        'HSBC',
        'BANCO AFIRME',
        'BANCO AZTECA',
        'BANCOPPEL',
        'SCOTIABANK',
        'BANCO REGIONAL DE MONTERREY',
        'BANCO DEL BAJIO',
    ]
}

In [156]:
from typing import Dict
from pathlib import Path

# Ingeniería de variables
from numpy import nan
from re import sub, UNICODE
from unicodedata import normalize
from datetime import datetime, date
from difflib import get_close_matches
from pandas import DataFrame, read_csv, to_datetime

class ShoppersChurn:
    def __init__(self, file_name: str='raw_data_MTY', correct_dict: Dict=correct_dict) -> None:
        self.correct_dict = correct_dict
        self.base_dir = Path.cwd().parent
        self.data_dir = self.base_dir.joinpath('data')
        self.file_name = file_name
        self.file_path = self.data_dir.joinpath(f'{self.file_name}.csv')
        if not self.file_path.is_file():
            print(f'There should be a file called "{self.file_name}" at:\n{self.data_dir}\n\nAdd it and try again!')


    def get_files(self, shopper_id_col: str='shopper_id') -> None:
        data = read_csv(self.file_path, low_memory=False)
        end_of_shopper_data = [x for x,y in enumerate(data.columns) if y=='end_of_shoppers_data'][0]
        self.sh = data.iloc[:,:end_of_shopper_data].drop_duplicates(shopper_id_col)
        self.df = data[[shopper_id_col]].join(data.iloc[:,end_of_shopper_data+1:])


    def clean_text(self, text: str, pattern: str="[^a-zA-Z0-9\s]", lower: bool=False) -> str: 
        '''
        Limpieza de texto
        '''
        # Reemplazar acentos: áàäâã --> a
        clean = normalize('NFD', str(text).replace('\n', ' \n ')).encode('ascii', 'ignore')
        # Omitir caracteres especiales !"#$%&/()=...
        clean = sub(pattern, ' ', clean.decode('utf-8'), flags=UNICODE)
        # Mantener sólo un espacio
        clean = sub(r'\s{2,}', ' ', clean.strip())
        # Minúsculas si el parámetro lo indica
        if lower: clean = clean.lower()
        # Si el registro estaba vacío, indicar nulo
        if clean in ('','nan'): clean = nan
        return clean


    def choose_correct(self, df: DataFrame, col: str, correct_list: list, fill_value: str='Otro', keep_nan: bool=True, replace_col: bool=True, **kwargs) -> DataFrame:
        '''
        Recibe un DataFrame y una lista de posibilidades, especificando la columna a revisar
        elige la opción que más se parezca a alguna de las posibilidades
        '''
        # Aplicar limpieza de texto a la lista de posibilidades
        correct_clean = list(map(lambda x: self.clean_text(x, lower=True), correct_list))+['nan']
        # Hacer un diccionario de posibilidades limpias y las originales recibidas
        correct_dict = dict(zip(correct_clean, correct_list+['nan']))

        # Aplicar la limpieza a la columna especificada
        df[f'{col}_correct'] = df[col].map(lambda x: self.clean_text(x,lower=True))
        # Encontrar las posibilidades más parecidas
        df[f'{col}_correct'] = df[f'{col}_correct'].map(lambda x: get_close_matches(str(x), correct_clean, **kwargs))
        # Si existen parecidas, traer la primera opción que es la más parecida
        df[f'{col}_correct'] = df[f'{col}_correct'].map(lambda x: x[0] if isinstance(x,list) and len(x)>0 else nan)
        # Regresar del texto limpio a la posibilidad original, lo no encontrado se llena con "fill_value"
        df[f'{col}_correct'] = df[f'{col}_correct'].map(correct_dict).fillna(fill_value)
        
        if keep_nan: df[f'{col}_correct'] = df[f'{col}_correct'].map(lambda x: nan if str(x)=='nan' else x)
        if replace_col: df = df.drop(col, axis=1).rename({f'{col}_correct':col}, axis=1)
        return df


    def clean_shopper_data(self, marital_col: str='marital_status', insurance_col: str='insurance', bank_col: str='bank', transport_col: str='transport') -> None:
        df = self.sh.copy()
        df[marital_col] = df[marital_col].map(lambda x: nan if str(x)=='nan' else x.replace(' ',''))

        aux = []
        for x in df[insurance_col]:
            if str(x)=='nan': aux.append(nan)
            else: 
                try: to_append = to_datetime(x, format=r'%d/%m/%y')
                except: 
                    try: to_append = to_datetime(x[:10], format=r'%Y-%m-%d')
                    except: 
                        try: to_append = to_datetime(x[:11], format=r'%d-%b-%Y')
                        except: to_append = nan
                finally: aux.append(to_append)
        df[insurance_col] = aux

        df = self.choose_correct(df, bank_col, self.correct_dict[bank_col], n=1, cutoff=0.7)

        df[transport_col] = df[transport_col].map(lambda x: nan if str(x)=='nan' else x.split()[0].title())
        aux = df[transport_col].value_counts(1).to_frame()
        self.correct_dict[transport_col] = [x for x,y in zip(aux.index, aux[transport_col]) if y>=0.02]
        df = self.choose_correct(df, transport_col, self.correct_dict[transport_col], n=1, cutoff=0.7)

        self.sh = df.copy()
        
    def vars_shopper(self, id_col: str='shopper_id', official_id_col: str='official_id', insurance_col: str='insurance', add_drop_cols: list=['last_date']) -> None:
        df = self.sh.set_index(id_col)

        df['birthday'] = to_datetime(df[official_id_col].str[4:10], format=r'%y%m%d')
        df['birthday'] = df['birthday'].map(lambda x: date(x.year-100, x.month, x.day) if x.year>datetime.today().year else x)
        df['age_in_days'] = (datetime.today() - df['birthday']).dt.days

        df['genre'] = df[official_id_col].str[10:11]

        df['days_for_insurance_exp'] = df[insurance_col].map(lambda x: nan if str(x)=='nan' else (x - datetime.today()).days)
        
        df.drop([official_id_col, 'birthday', insurance_col]+add_drop_cols, axis=1, inplace=True)

        self.shop_num_cols = df.sample(frac=0.01).describe().columns
        self.shop_cat_cols = [x for x in df.columns if x not in self.shop_num_cols]
        self.sh = df.copy()

sc = ShoppersChurn()
sc.get_files()
sc.clean_shopper_data()
sc.vars_shopper()

In [158]:
sc.shop_num_cols

Index(['pickings_vs_deliveries', 'completed_deliveries', 'completed_pickings',
       'completed_pickings_score', 'rating', 'accepted_rate',
       'accepted_rate_score', 'fulfillment_rate', 'picking_speed',
       'late_minutes_score', 'refund_post_sales_score', 'replacements_score',
       'call_chat_score', 'score', 'age_in_days', 'days_for_insurance_exp'],
      dtype='object')